# Model for baseline conditions only

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

#### For Attention List:
1. Need to go through and check the units work properly. Note that the values didnt change when I removed units so seems okay
2. Check units on flows
3. Check units on Vt
4. Add units to to all columns and decide upon my convention for this
5. Check units and speak to Payne about the wall thickness calcs
6. Is velocity redundant?  
*TISSUE PARTIALS ARE DEFINITELY WRONG*

#### Set Constants and paramaters

In [8]:
#input params
H = 0.42 #no units,ratio #Hematocrit assumed to be constant
M = 30* 10e-9 #mol_ O2/(mL/s) #Taken from table 2 from Wiley Payne paper
cHb = 0.2 #mL_O2/m #Taken from table 2 from Wiley Payne paper
paO2_bar_t = 15 #mmHG #Taken from table 2 from Wiley Payne paper
K = 5*10e-8

##reset df
df = None

#Input data from paper
df = pd.DataFrame({'Name': ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'C', 'V6', 'V5', 'V4', 'V3', 'V2', 'V1'], 
                   'Number': [1, 2, 4, 8, 16, 32, 64, 32, 16, 8, 4, 2, 1],
                   'Diameter(µm)': [23.97, 19.17, 15.28, 12.08, 9.46, 7.32, 8, 11.51, 14.53, 17.79, 21.45, 25.70, 30.77],
                   'Length(µm)': [1267.6, 930.3, 543.6, 302.3, 161.2, 154.7, 243.9, 473.9, 272.3, 426.6, 632.5, 844.2, 936.3],
                   'Viscocity(mPAs)': [1.59, 1.50, 1.42, 1.34, 1.28, 1.23, 1.24, 1.33, 1.40, 1.48, 1.55, 1.62, 1.7],
                   'Pressure Drop(mmHg)': [6.93, 5.87, 4.02, 2.70, 1.82, 2.35, 2.62, 1.27, 0.61, 0.89, 1.31, 1.78, 2.01],
                   'Saturation': [0.94, 0.93, 0.92, 0.89, 0.84, 0.765, 0.665, 0.61, 0.5975, 0.5875, 0.5825, 0.5775, 0.5725]
                  })


#### Calculate Flows and Vt

In [9]:
#Calculate resistances of each vessel
df['Resistance for U'] = 32 * df['Viscocity(mPAs)'] * df['Length(µm)'].div(df['Diameter(µm)']**2)
df['Resistance for Q'] = 128 * df['Viscocity(mPAs)'] * df['Length(µm)'].div(math.pi*(df['Diameter(µm)']**4))

#Alter resistances for C
#These are aross each pressure drop (explains the doubled resistance for the capillaries as in paralell so have half the flow each (flow is shared))
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for U']*2
df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q'] = df.loc[df.index[df['Name'] == 'C'][0],'Resistance for Q']*2

#Calculate flow and velocity in a single vessel
df['U in single(m/s????)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for U'])
df['Q in single(m^3/s????)'] = df['Pressure Drop(mmHg)'].div(df['Resistance for Q'])

#Calculate Vt
df['Vt'] = df['Q in single(m^3/s????)']*cHb*H*df['Saturation']/M

#### Calculate partial pressure in blood
S and partial O2 relationship given by servinghaus 1979  
Checked partial pressure against typical values online and it seems bang on.

In [15]:
def check_imag_roots_real(all_roots):
    counter = 0
    sols = 0
    for i in range(len(all_roots)):
        if all_roots[i].imag <= 10e-13 and all_roots[i].imag >= -10e-13:
            sols = all_roots[i].real
            counter += 1
    if counter != 1:
        print('Something seems wrong with the partial presure to Saturation cubic solver as not one solution is returned.')
    return sols
 
for i in range(len(df['Saturation'])):
    root_temp = None
    sols = 0
    root_temp = np.roots([1,0,150,23400*df.loc[i,'Saturation']/(df.loc[i,'Saturation']-1)]) #gives mmHg , *133.322
    sols = check_imag_roots_real(root_temp) 
    #sols = sols*133.322 ###Remove this step if want in mmHg
    df.loc[i,'partial pressure blood(mmHg)'] = sols

#### Calculate wall thickness
Will take my vessel average partial pressure to be the 0.5(pin + pout)  
All units were in mm so i played around with this equation from the paper  
Equations are taken from Payne paper

In [11]:
#all units were in mm so i played around with this equation from the paper
df['X-Area wall(mm^2?)'] = np.pi * ( 0.16*(df['Diameter(µm)'])**2 + 1.4*(df['Diameter(µm)']) + 14 )
df['wall thickness(µm)'] = ( -(df['Diameter(µm)']) + np.sqrt((df['Diameter(µm)'])**2 + (4*(df['X-Area wall(mm^2?)'])/np.pi)) ) / 2

#### Calculate partial pressure in tissue
Equations can be found on the week 1 or 2 notes from class. From steady state version of something on the paper

In [12]:
df['tissue partials(mmHg)'] = df['partial pressure blood(mmHg)'] - M * df['Vt'] * df['wall thickness(µm)'] / (2 * np.pi * K * (df['Diameter(µm)']/2) * df['Length(µm)'])

### SHOW RESULTS

In [14]:
df

,Name,Number,Diameter(µm),Length(µm),Viscocity(mPAs),Pressure Drop(mmHg),Saturation,Resistance for U,Resistance for Q,U in single(m/s????),Q in single(m^3/s????),Vt,partial pressure blood(mmHg),X-Area wall(mm^2?),wall thickness(µm),tissue partials(mmHg)
0,A1,1,23.97,1267.6,1.59,6.93,0.9400,112.251787,0.248752,0.061736,27.859025,7.332495e+06,70.871373,438.213672,4.841407,-152.267123
1,A2,2,19.17,930.3,1.50,5.87,0.9300,121.512242,0.421004,0.048308,13.942870,3.630723e+06,67.005340,313.016429,4.253658,-98.385688
2,A3,4,15.28,543.6,1.42,4.02,0.9200,105.796442,0.576945,0.037997,6.967734,1.794888e+06,63.786724,228.546291,3.810683,-93.480728
3,A4,8,12.08,302.3,1.34,2.70,0.8900,88.829876,0.775060,0.030395,3.483601,8.681135e+05,56.550348,170.463521,3.485850,-101.713429
4,A5,16,9.46,161.2,1.28,1.82,0.8400,73.780690,1.049713,0.024668,1.733807,4.077915e+05,48.706014,130.572935,3.265971,-118.093855
5,A6,32,7.32,154.7,1.23,2.35,0.7650,113.637911,2.700295,0.020680,0.870275,1.864130e+05,41.211431,103.110791,3.138293,-57.455033
6,C,64,8.00,243.9,1.24,2.62,0.6650,302.436000,6.016773,0.008663,0.435449,8.108067e+04,34.556920,111.338044,3.172168,9.381706
7,V6,32,11.51,473.9,1.33,1.27,0.6100,152.243122,1.463178,0.008342,0.867974,1.482499e+05,31.696836,161.197683,3.433626,13.873593
8,V5,16,14.53,272.3,1.40,0.61,0.5975,57.782247,0.348476,0.010557,1.750477,2.928549e+05,31.097186,214.009514,3.730525,-21.639220
9,V4,8,17.79,426.6,1.48,0.89,0.5875,63.838202,0.256826,0.013941,3.465382,5.700554e+05,30.628584,281.309064,4.092089,-28.075280
